# Batch Normalization
:label:`sec_batch_norm`

Training deep neural nets is difficult.
And getting them to converge in a reasonable amount of time can be tricky.
In this section, we describe batch normalization (BN)
:cite:`Ioffe.Szegedy.2015`, a popular and effective technique
that consistently accelerates the convergence of deep nets.
Together with residual blocks—covered in :numref:`sec_resnet`—BN
has made it possible for practitioners
to routinely train networks with over 100 layers.



## Training Deep Networks

To motivate batch normalization, let us review
a few practical challenges that arise
when training ML models and neural nets in particular.

1. Choices regarding data preprocessing often
   make an enormous difference in the final results.
   Recall our application of multilayer perceptrons
   to predicting house prices (:numref:`sec_kaggle_house`).
   Our first step when working with real data
   was to standardize our input features
   to each have a mean of *zero* and variance of *one*.
   Intuitively, this standardization plays nicely with our optimizers
   because it puts the parameters a-priori at a similar scale.
2. For a typical MLP or CNN, as we train,
   the activations in intermediate layers
   may take values with widely varying magnitudes—both
   along the layers from the input to the output,
   across nodes in the same layer,
   and over time due to our updates to the model's parameters.
   The inventors of batch normalization postulated informally
   that this drift in the distribution of activations
   could hamper the convergence of the network.
   Intuitively, we might conjecture that if one
   layer has activation values that are 100x that of another layer,
   this might necessitate compensatory adjustments in the learning rates.
3. Deeper networks are complex and easily capable of overfitting.
   This means that regularization becomes more critical.

Batch normalization is applied to individual layers
(optionally, to all of them) and works as follows:
In each training iteration,
we first normalize the inputs (of batch normalization)
by subtracting their mean and
dividing by their standard deviation,
where both are estimated based on the statistics of the current minibatch.
Next, we apply a scaling coefficient and a scaling offset.
It is precisely due to this *normalization* based on *batch* statistics
that *batch normalization* derives its name.

Note that if we tried to apply BN with minibatches of size $1$,
we would not be able to learn anything.
That is because after subtracting the means,
each hidden node would take value $0$!
As you might guess, since we are devoting a whole section to BN,
with large enough minibatches, the approach proves effective and stable.
One takeaway here is that when applying BN,
the choice of minibatch size may be
even more significant than without BN.

Formally, BN transforms the activations at a given layer $\mathbf{x}$
according to the following expression:

$$\mathrm{BN}(\mathbf{x}) = \mathbf{\gamma} \odot \frac{\mathbf{x} - \hat{\mathbf{\mu}}}{\hat\sigma} + \mathbf{\beta}$$

Here, $\hat{\mathbf{\mu}}$ is the minibatch sample mean
and $\hat{\mathbf{\sigma}}$ is the minibatch sample standard deviation.
After applying BN, the resulting minibatch of activations
has zero mean and unit variance.
Because the choice of unit variance
(vs some other magic number) is an arbitrary choice,
we commonly include coordinate-wise
scaling coefficients $\mathbf{\gamma}$ and offsets $\mathbf{\beta}$.
Consequently, the activation magnitudes
for intermediate layers cannot diverge during training
because BN actively centers and rescales them back
to a given mean and size (via $\mathbf{\mu}$ and $\sigma$).
One piece of practitioner's intuition/wisdom
is that BN seems to allows for more aggressive learning rates.


Formally, denoting a particular minibatch by $\mathcal{B}$,
we calculate $\hat{\mathbf{\mu}}_\mathcal{B}$ and $\hat\sigma_\mathcal{B}$ as follows:

$$\hat{\mathbf{\mu}}_\mathcal{B} \leftarrow \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x}
\text{ and }
\hat{\mathbf{\sigma}}_\mathcal{B}^2 \leftarrow \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \mathbf{\mu}_{\mathcal{B}})^2 + \epsilon$$

Note that we add a small constant $\epsilon > 0$
to the variance estimate
to ensure that we never attempt division by zero,
even in cases where the empirical variance estimate might vanish.
The estimates $\hat{\mathbf{\mu}}_\mathcal{B}$
and $\hat{\mathbf{\sigma}}_\mathcal{B}$ counteract the scaling issue
by using noisy estimates of mean and variance.
You might think that this noisiness should be a problem.
As it turns out, this is actually beneficial.

This turns out to be a recurring theme in deep learning.
For reasons that are not yet well-characterized theoretically,
various sources of noise in optimization
often lead to faster training and less overfitting.
While traditional machine learning theorists
might buckle at this characterization,
this variation appears to act as a form of regularization.
In some preliminary research,
:cite:`Teye.Azizpour.Smith.2018` and :cite:`Luo.Wang.Shao.ea.2018`
relate the properties of BN to Bayesian Priors and penalties respectively.
In particular, this sheds some light on the puzzle
of why BN works best for moderate minibatches sizes in the $50$–$100$ range.

Fixing a trained model, you might (rightly) think
that we would prefer to use the entire dataset
to estimate the mean and variance.
Once training is complete, why would we want
the same image to be classified differently,
depending on the batch in which it happens to reside?
During training, such exact calculation is infeasible
because the activations for all data points
change every time we update our model.
However, once the model is trained,
we can calculate the means and variances
of each layer's activations based on the entire dataset.
Indeed this is standard practice for
models employing batch normalization
and thus BN layers function differently
in *training mode* (normalizing by minibatch statistics)
and in *prediction mode* (normalizing by dataset statistics).

We are now ready to take a look at how batch normalization works in practice.


## Batch Normalization Layers

Batch normalization implementations for fully-connected layers
and convolutional layers are slightly different.
We discuss both cases below.
Recall that one key differences between BN and other layers
is that because BN operates on a full minibatch at a time,
we cannot just ignore the batch dimension
as we did before when introducing other layers.


### Fully-Connected Layers

When applying BN to fully-connected layers,
we usually insert BN after the affine transformation
and before the nonlinear activation function.
Denoting the input to the layer by $\mathbf{x}$,
the linear transform (with weights $\theta$) by $f_{\theta}(\cdot)$,
the activation function by $\phi(\cdot)$,
and the BN operation with parameters $\mathbf{\beta}$ and $\mathbf{\gamma}$
by $\mathrm{BN}_{\mathbf{\beta}, \mathbf{\gamma}}$,
we can express the computation of a BN-enabled,
fully-connected layer $\mathbf{h}$ as follows:

$$\mathbf{h} = \phi(\mathrm{BN}_{\mathbf{\beta}, \mathbf{\gamma}}(f_{\mathbf{\theta}}(\mathbf{x}) ) ) $$

Recall that mean and variance are computed
on the *same* minibatch $\mathcal{B}$
on which the transformation is applied.
Also recall that the scaling coefficient $\mathbf{\gamma}$
and the offset $\mathbf{\beta}$ are parameters that need to be learned
jointly with the more familiar parameters $\mathbf{\theta}$.

### Convolutional Layers

Similarly, with convolutional layers,
we typically apply BN after the convolution
and before the nonlinear activation function.
When the convolution has multiple output channels,
we need to carry out batch normalization
for *each* of the outputs of these channels,
and each channel has its own scale and shift parameters,
both of which are scalars.
Assume that our minibatches contain $m$ each
and that for each channel,
the output of the convolution has height $p$ and width $q$.
For convolutional layers, we carry out each batch normalization
over the $m \cdot p \cdot q$ elements per output channel simultaneously.
Thus we collect the values over all spatial locations
when computing the mean and variance
and consequently (within a given channel)
apply the same $\hat{\mathbf{\mu}}$ and $\hat{\mathbf{\sigma}}$
to normalize the values at each spatial location.


### Batch Normalization During Prediction

As we mentioned earlier, BN typically behaves differently
in training mode and prediction mode.
First, the noise in $\mathbf{\mu}$ and $\mathbf{\sigma}$
arising from estimating each on minibatches
are no longer desirable once we have trained the model.
Second, we might not have the luxury
of computing per-batch normalization statistics, e.g.,
we might need to apply our model to make one prediction at a time.

Typically, after training, we use the entire dataset
to compute stable estimates of the activation statistics
and then fix them at prediction time.
Consequently, BN behaves differently during training and at test time.
Recall that dropout also exhibits this characteristic.

## Implementation from Scratch

Below, firstly we get all the relevant libraries needed to implement BatchNorm. After that, we implement a batch normalization layer with NDArrays from scratch:


In [21]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.util.Training

fun getLong(nm: String, n: Long): Long {
    val name = System.getProperty(nm)
    return if (null == name) n.toLong() else name.toLong()
}


In [22]:
import ai.djl.basicdataset.cv.classification.*;
// import org.apache.commons.lang3.ArrayUtils;

In [23]:
    fun batchNormUpdate(
        X: NDArray,
        gamma: NDArray,
        beta: NDArray,
        movingMean0: NDArray,
        movingVar0: NDArray,
        eps: Float,
        momentum: Float,
        isTraining: Boolean
    ): NDList {
        // attach moving mean and var to submanager to close intermediate computation values
        // at the end to avoid memory leak
        var movingMean = movingMean0
        var movingVar = movingVar0
        movingMean.manager.newSubManager().use { subManager ->
            movingMean.attach(subManager)
            movingVar.attach(subManager)
            val xHat: NDArray
            val mean: NDArray
            val vari: NDArray
            if (!isTraining) {
                // If it is the prediction mode, directly use the mean and variance
                // obtained from the incoming moving average
                xHat = X.sub(movingMean).div(movingVar.add(eps).sqrt())
            } else {
                if (X.shape.dimension() == 2) {
                    // When using a fully connected layer, calculate the mean and
                    // variance on the feature dimension
                    mean = X.mean(intArrayOf(0), true)
                    vari = X.sub(mean).pow(2).mean(intArrayOf(0), true)
                } else {
                    // When using a two-dimensional convolutional layer, calculate the
                    // mean and variance on the channel dimension (axis=1). Here we
                    // need to maintain the shape of `X`, so that the broadcast
                    // operation can be carried out later
                    mean = X.mean(intArrayOf(0, 2, 3), true)
                    vari = X.sub(mean).pow(2).mean(intArrayOf(0, 2, 3), true)
                }
                // In training mode, the current mean and variance are used for the
                // standardization
                xHat = X.sub(mean).div(vari.add(eps).sqrt())
                // Update the mean and variance of the moving average
                movingMean = movingMean.mul(momentum).add(mean.mul(1.0f - momentum))
                movingVar = movingVar.mul(momentum).add(vari.mul(1.0f - momentum))
            }
            val Y = xHat.mul(gamma).add(beta) // Scale and shift
            // attach moving mean and var back to original manager to keep their values
            movingMean.attach(subManager.parentManager)
            movingVar.attach(subManager.parentManager)
            return NDList(Y, movingMean, movingVar)
        }
    }


We can now create a proper `BatchNorm` layer. Our layer will maintain proper parameters corresponding for scale gamma and shift beta, both of which will be updated in the course of training. Additionally, our layer will maintain a moving average of the means and variances for subsequent use during model prediction. The numFeatures parameter required by the BatchNorm instance is the number of outputs for a fully-connected layer and the number of output channels for a convolutional layer. The numDimensions parameter also required by this instance is 2 for a fully-connected layer and 4 for a convolutional layer.

Putting aside the algorithmic details, note the design pattern underlying our implementation of the layer. Typically, we define the math in a separate function, say `batchNormUpdate`. We then integrate this functionality into a custom layer, whose code mostly addresses bookkeeping matters, such as moving data to the right device context, allocating and initializing any required variables, keeping track of running averages (here for mean and variance), etc. This pattern enables a clean separation of math from boilerplate code. Also note that for the sake of convenience we did not worry about automatically inferring the input shape here, thus we need to specify the number of features throughout. Do not worry, the DJL `BatchNorm` layer will care of this for us.


In [24]:
class BatchNormBlock(numFeatures: Int, numDimensions: Int) : AbstractBlock() {

    private var movingMean: NDArray
    private var movingVar: NDArray
    private var gamma: Parameter
    private var beta: Parameter
    private var shape: Shape

    // num_features: the number of outputs for a fully-connected layer
    // or the number of output channels for a convolutional layer.
    // num_dims: 2 for a fully-connected layer and 4 for a convolutional layer.
    init {
        if (numDimensions == 2) {
            shape = Shape(1, numFeatures.toLong())
        } else {
            shape = Shape(1, numFeatures.toLong(), 1, 1)
        }
        // The scale parameter and the shift parameter involved in gradient
        // finding and iteration are initialized to 0 and 1 respectively
        gamma = addParameter(
            Parameter.builder()
                .setName("gamma")
                .setType(Parameter.Type.GAMMA)
                .optShape(shape)
                .build()
        )
        beta = addParameter(
            Parameter.builder()
                .setName("beta")
                .setType(Parameter.Type.BETA)
                .optShape(shape)
                .build()
        )

        // All the variables not involved in gradient finding and iteration are
        // initialized to 0. Create a base manager to maintain their values
        // throughout the entire training process
        val manager = NDManager.newBaseManager()
        movingMean = manager.zeros(shape)
        movingVar = manager.zeros(shape)
    }
    override fun toString(): String {
        return "jp.live.ugai.d2j.BatchNormBlock()"
    }

    override fun forwardInternal(
        parameterStore: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        val result = batchNormUpdate(
            inputs.singletonOrThrow(),
            gamma.array,
            beta.array,
            movingMean,
            movingVar,
            1e-12f,
            0.9f,
            training
        )
        // close previous NDArray before assigning new values
        if (training) {
            movingMean.close()
            movingVar.close()
        }
        // Save the updated `movingMean` and `movingVar`
        movingMean = result[1]
        movingVar = result[2]
        return NDList(result[0])
    }

    override fun getOutputShapes(inputs: Array<Shape>): Array<Shape> {
        var current = inputs
        for (block in children.values()) {
            current = block.getOutputShapes(current)
        }
        return current
    }
}


## Using a Batch Normalization LeNet

To see how to apply `BatchNorm` in context,
below we apply it to a traditional LeNet model (:numref:`sec_lenet`).
Recall that BN is typically applied
after the convolutional layers and fully-connected layers
but before the corresponding activation functions.


In [25]:
val block: SequentialBlock = SequentialBlock()
        .add(
            Conv2d.builder()
                .setKernelShape(Shape(5, 5))
                .setFilters(6).build()
        )
        .add(BatchNorm.builder().build())
        .add(Pool.maxPool2dBlock(Shape(2, 2), Shape(2, 2)))
        .add(
            Conv2d.builder()
                .setKernelShape(Shape(5, 5))
                .setFilters(16).build()
        )
        .add(BatchNorm.builder().build())
        .add(Activation::sigmoid)
        .add(Pool.maxPool2dBlock(Shape(2, 2), Shape(2, 2)))
        .add(Blocks.batchFlattenBlock())
        .add(Linear.builder().setUnits(120).build())
        .add(BatchNorm.builder().build())
        .add(Activation::sigmoid)
        .add(Blocks.batchFlattenBlock())
        .add(Linear.builder().setUnits(84).build())
        .add(BatchNorm.builder().build())
        .add(Activation::sigmoid)
        .add(Linear.builder().setUnits(10).build())


Let's initialize the batchSize, numEpochs and the relevant arrays to store the data from the training function.

In [26]:
    val batchSize = 256
    val numEpochs = Integer.getInteger("MAX_EPOCH", 10)
//epochCount = new double[numEpochs];
//
//for (int i = 0; i < epochCount.length; i++) {
//    epochCount[i] = i+1;
//}

As before, we will train our network on the Fashion-MNIST dataset.
This code is virtually identical to that when we first trained LeNet (:numref:`sec_lenet`).
The main difference is the considerably larger learning rate.


In [27]:
    val trainIter = FashionMnist.builder()
        .optUsage(Dataset.Usage.TRAIN)
        .setSampling(batchSize, true)
        .optLimit(getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build()

    val testIter = FashionMnist.builder()
        .optUsage(Dataset.Usage.TEST)
        .setSampling(batchSize, true)
        .optLimit(getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build()

    trainIter.prepare()
    testIter.prepare()


In [28]:
    val loss: Loss = Loss.softmaxCrossEntropyLoss()

    val lrt: Tracker = Tracker.fixed(1.0f)
    val sgd: Optimizer = Optimizer.sgd().setLearningRateTracker(lrt).build()

    val model: Model = Model.newInstance("batch-norm")
    model.block = block

    val config = DefaultTrainingConfig(loss)
        .optOptimizer(sgd) // Optimizer (loss function)
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()) // Logging

    val trainer: Trainer = model.newTrainer(config)
    trainer.initialize(Shape(1, 1, 28, 28))

    val evaluatorMetrics: MutableMap<String, DoubleArray> = mutableMapOf()
    val avgTrainTimePerEpoch = Training.trainingChapter6(trainIter, testIter, numEpochs, trainer, evaluatorMetrics)

Training:    100% |████████████████████████████████████████| Accuracy: 0.79, SoftmaxCrossEntropyLoss: 0.61
Validating:  100% |████████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.86, SoftmaxCrossEntropyLoss: 0.39
Validating:  100% |████████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.88, SoftmaxCrossEntropyLoss: 0.35
Validating:  100% |████████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.88, SoftmaxCrossEntropyLoss: 0.33
Validating:  100% |████████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.31
Validating:  100% |████████████████████████████████████████|
Training:     31% |███████�█████                           | Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.29

In [33]:
val trainLoss = evaluatorMetrics.get("train_epoch_SoftmaxCrossEntropyLoss");
val trainAccuracy = evaluatorMetrics.get("train_epoch_Accuracy");
val testAccuracy = evaluatorMetrics.get("validate_epoch_Accuracy");

print("loss %.3f,".format(trainLoss!![numEpochs - 1]))
print(" train acc %.3f,".format(trainAccuracy!![numEpochs - 1]))
print(" test acc %.3f\n".format(testAccuracy!![numEpochs - 1]))
print("%.1f examples/sec".format(trainIter.size() / (avgTrainTimePerEpoch / Math.pow(10.0, 9.0))))
println()

loss 0.256, train acc 0.907, test acc 0.828
1976.4 examples/sec


Let us have a look at the scale parameter `gamma`
and the shift parameter `beta` learned
from the first batch normalization layer.


In [35]:
// Printing the value of gamma and beta in the first BatchNorm layer.
val batchNormFirstParams = block.getChildren().values().get(1).getParameters().values();
println("gamma " + batchNormFirstParams.get(0).getArray().reshape(-1));
println("beta " + batchNormFirstParams.get(1).getArray().reshape(-1));

gamma ND: (6) cpu() float32
[1.2309, 1.3872, 1.3243, 1.1022, 1.5128, 1.2874]

beta ND: (6) cpu() float32
[-2.14630791e-06, -1.02788817e-06,  9.31525847e-07, -2.22200924e-07, -1.82536496e-07,  9.03508294e-07]



![Contour Gradient Descent.](https://d2l-java-resources.s3.amazonaws.com/img/chapter_convolution-modern-cnn-batchnorm1.png)

In [40]:
//    trainLoss = evaluatorMetrics.get("train_epoch_SoftmaxCrossEntropyLoss");
//    trainAccuracy = evaluatorMetrics.get("train_epoch_Accuracy");
//    testAccuracy = evaluatorMetrics.get("validate_epoch_Accuracy");

    print("loss %.3f,".format(trainLoss[numEpochs-1]))
    print(" train acc %.3f,".format(trainAccuracy[numEpochs-1]))
    print(" test acc %.3f\n".format(testAccuracy[numEpochs-1]))
    print("%.1f examples/sec \n".format(trainIter.size() / (avgTrainTimePerEpoch / Math.pow(10.toDouble(), 9.0))))
    val epochCount = IntArray(numEpochs) { it } 
    val labels = Array<String>(numEpochs) { "train loss"} + Array<String>(numEpochs) { "train Acc"} + Array<String>(numEpochs) { "test Acc"}
    val data = mapOf("x" to epochCount + epochCount + epochCount,
                    "y" to trainLoss!! + trainAccuracy!! + testAccuracy!!,
                    "color" to labels)
    var plot = letsPlot(data)
    plot += geomLine { x = "x"; y="y"; color ="color"}
    plot+ggsize(700,500)

loss 0.256, train acc 0.907, test acc 0.828
1976.4 examples/sec 


## Concise Implementation

Compared with the `BatchNorm` class, which we just defined ourselves, the `BatchNorm` class defined by `nn` module in DJL is easier to use. In DJL, we do not have to worry about `numFeatures` or `numDimensions`. Instead, these parameter values will be inferred automatically via delayed initialization. Otherwise, the code looks virtually identical to the application our implementation above.


In [41]:
val block = SequentialBlock()
    .add(
            Conv2d.builder()
                    .setKernelShape(Shape(5, 5))
                    .setFilters(6).build())
    .add(BatchNorm.builder().build())
    .add(Pool.maxPool2dBlock(Shape(2, 2), Shape(2, 2)))
    .add(
            Conv2d.builder()
                    .setKernelShape(Shape(5, 5))
                    .setFilters(16).build())
    .add(BatchNorm.builder().build())
    .add(Activation::sigmoid)
    .add(Pool.maxPool2dBlock(Shape(2, 2), Shape(2, 2)))
    .add(Blocks.batchFlattenBlock())
    .add(Linear.builder().setUnits(120).build())
    .add(BatchNorm.builder().build())
    .add(Activation::sigmoid)
    .add(Blocks.batchFlattenBlock())
    .add(Linear.builder().setUnits(84).build())
    .add(BatchNorm.builder().build())
    .add(Activation::sigmoid)
    .add(Linear.builder().setUnits(10).build());

Below, we use the same hyperparameters to train out model.
Note that as usual, the high-level API variant runs much faster
because its code has been compiled to C++/CUDA
while our custom implementation must be interpreted by Python.


In [43]:
val loss = Loss.softmaxCrossEntropyLoss();

val lrt = Tracker.fixed(1.0f);
val sgd = Optimizer.sgd().setLearningRateTracker(lrt).build();

val model = Model.newInstance("batch-norm");
model.setBlock(block);

val config = DefaultTrainingConfig(loss)
                .optOptimizer(sgd) // Optimizer (loss function)
                .addEvaluator(Accuracy()) // Model Accuracy
                .addTrainingListeners(*TrainingListener.Defaults.logging()); // Logging

val trainer = model.newTrainer(config);
trainer.initialize(Shape(1, 1, 28, 28));

// val evaluatorMetrics = new HashMap<>();
val evaluatorMetrics = mutableMapOf<String, DoubleArray>()
var avgTrainTimePerEpoch = 0.0

In [44]:
avgTrainTimePerEpoch = Training.trainingChapter6(trainIter, testIter, numEpochs, trainer, evaluatorMetrics);

Training:    100% |████████████████████████████████████████| Accuracy: 0.79, SoftmaxCrossEntropyLoss: 0.61
Validating:  100% |████████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.86, SoftmaxCrossEntropyLoss: 0.39
Validating:  100% |████████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.87, SoftmaxCrossEntropyLoss: 0.35
Validating:  100% |████████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.88, SoftmaxCrossEntropyLoss: 0.32
Validating:  100% |████████████████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.31
Validating:  100% |████████████████████████████████████████|
Training:     31% |███████�█████                           | Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.29

In [45]:
val trainLoss = evaluatorMetrics.get("train_epoch_SoftmaxCrossEntropyLoss");
val trainAccuracy = evaluatorMetrics.get("train_epoch_Accuracy");
val testAccuracy = evaluatorMetrics.get("validate_epoch_Accuracy");

print("loss %.3f,".format(trainLoss!![numEpochs - 1]))
print(" train acc %.3f,".format(trainAccuracy!![numEpochs - 1]))
print(" test acc %.3f\n".format(testAccuracy!![numEpochs - 1]))
print("%.1f examples/sec".format(trainIter.size() / (avgTrainTimePerEpoch / Math.pow(10.0, 9.0))))
println()

loss 0.253, train acc 0.907, test acc 0.826
1973.1 examples/sec


![Contour Gradient Descent.](https://d2l-java-resources.s3.amazonaws.com/img/chapter_convolution-modern-cnn-batchnorm2.png)

In [46]:
    print("loss %.3f,".format(trainLoss[numEpochs-1]))
    print(" train acc %.3f,".format(trainAccuracy[numEpochs-1]))
    print(" test acc %.3f\n".format(testAccuracy[numEpochs-1]))
    print("%.1f examples/sec \n".format(trainIter.size() / (avgTrainTimePerEpoch / Math.pow(10.toDouble(), 9.0))))
    val epochCount = IntArray(numEpochs) { it } 
    val labels = Array<String>(numEpochs) { "train loss"} + Array<String>(numEpochs) { "train Acc"} + Array<String>(numEpochs) { "test Acc"}
    val data = mapOf("x" to epochCount + epochCount + epochCount,
                    "y" to trainLoss!! + trainAccuracy!! + testAccuracy!!,
                    "color" to labels)
    var plot = letsPlot(data)
    plot += geomLine { x = "x"; y="y"; color ="color"}
    plot+ggsize(700,500)

loss 0.253, train acc 0.907, test acc 0.826
1973.1 examples/sec 


## Controversy

Intuitively, batch normalization is thought
to make the optimization landscape smoother.
However, we must be careful to distinguish between
speculative intuitions and true explanations
for the phenomena that we observe when training deep models.
Recall that we do not even know why simpler
deep neural networks (MLPs and conventional CNNs)
generalize well in the first place.
Even with dropout and $L_2$ regularization,
they remain so flexible that their ability to generalize to unseen data
cannot be explained via conventional learning-theoretic generalization guarantees.

In the original paper proposing batch normalization,
the authors, in addition to introducing a powerful and useful tool,
offered an explanation for why it works:
by reducing *internal covariate shift*.
Presumably by *internal covariate shift* the authors
meant something like the intuition expressed above—the
notion that the distribution of activations changes
over the course of training.
However there were two problems with this explanation:
(1) This drift is very different from *covariate shift*,
rendering the name a misnomer.
(2) The explanation offers an under-specified intuition
but leaves the question of *why precisely this technique works*
an open question wanting for a rigorous explanation.
Throughout this book, we aim to convey the intuitions that practitioners
use to guide their development of deep neural networks.
However, we believe that it is important
to separate these guiding intuitions
from established scientific fact.
Eventually, when you master this material
and start writing your own research papers
you will want to be clear to delineate
between technical claims and hunches.

Following the success of batch normalization,
its explanation in terms of *internal covariate shift*
has repeatedly surfaced in debates in the technical literature
and broader discourse about how to present machine learning research.
In a memorable speech given while accepting a Test of Time Award
at the 2017 NeurIPS conference,
Ali Rahimi used *internal covariate shift*
as a focal point in an argument likening
the modern practice of deep learning to alchemy.
Subsequently, the example was revisited in detail
in a position paper outlining
troubling trends in machine learning :cite:`Lipton.Steinhardt.2018`.
In the technical literature other authors (:cite:`Santurkar.Tsipras.Ilyas.ea.2018`)
have proposed alternative explanations for the success of BN,
some claiming that BN's success comes despite exhibiting behavior
that is in some ways opposite to those claimed in the original paper.

We note that the *internal covariate shift*
is no more worthy of criticism than any of
thousands of similarly vague claims
made every year in the technical ML literature.
Likely, its resonance as a focal point of these debates
owes to its broad recognizability to the target audience.
Batch normalization has proven an indispensable method,
applied in nearly all deployed image classifiers,
earning the paper that introduced the technique
tens of thousands of citations.


## Summary

* During model training, batch normalization continuously adjusts the intermediate output of the neural network by utilizing the mean and standard deviation of the minibatch, so that the values of the intermediate output in each layer throughout the neural network are more stable.
* The batch normalization methods for fully connected layers and convolutional layers are slightly different.
* Like a dropout layer, batch normalization layers have different computation results in training mode and prediction mode.
* Batch Normalization has many beneficial side effects, primarily that of regularization. On the other hand, the original motivation of reducing covariate shift seems not to be a valid explanation.

## Exercises

1. Can we remove the fully connected affine transformation before the batch normalization or the bias parameter in convolution computation?
    * Find an equivalent transformation that applies prior to the fully connected layer.
    * Is this reformulation effective. Why (not)?
1. Compare the learning rates for LeNet with and without batch normalization.
    * Plot the decrease in training and test error.
    * What about the region of convergence? How large can you make the learning rate?
1. Do we need Batch Normalization in every layer? Experiment with it?
1. Can you replace Dropout by Batch Normalization? How does the behavior change?
1. Fix the coefficients `beta` and `gamma` , and observe and analyze the results.
1. Review the online documentation for `BatchNorm` to see the other applications for Batch Normalization.
1. Research ideas: think of other normalization transforms that you can apply? Can you apply the probability integral transform? How about a full rank covariance estimate?
